<a href="https://colab.research.google.com/github/e-olang/NLP/blob/main/Q%26A/QA_Draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers datasets evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.9 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset

dataset = load_dataset("masakhane/afriqa", "swa")
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 415
    })
    validation: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 417
    })
    test: Dataset({
        features: ['question', 'answers', 'lang', 'split', 'translated_question', 'translated_answer', 'translation_type'],
        num_rows: 302
    })
})

In [9]:
dataset['train']['question'][:5]

['Je,mkoa wa Arusha ina idadi ya watu wangapi?',
 'Je,Thiago Silva alichezea Brazil katika kombe la dunia mwaka 2018?',
 'Sun Yat-sen alikuwa ameoa?',
 'Je,Wamaragoli huzungumza lugha gani?',
 'Gavana wa kwanza kaunti ya Marsabit ni nani?']

In [10]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("eolang/SW-v1")

In [14]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [12]:
dataset["train"].column_names

['question',
 'answers',
 'lang',
 'split',
 'translated_question',
 'translated_answer',
 'translation_type']

In [ ]:
['question',
 'answers'
 ]

In [15]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns = dataset["train"].column_names)

Map:   0%|          | 0/415 [00:00<?, ? examples/s]

TypeError: ignored